# Comparision of 2 versions of Ameriflux data

In [74]:
import sys
sys.path.append( '/home/greg/current/NMEG_utils/py_modules/' )

%matplotlib
import load_nmeg as ld
import transform_nmeg as tr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np

# Years to load
start = 2007
end = 2014


Using matplotlib backend: Qt4Agg


## Hourly data

In [108]:
# Load for one site
site = 'Vcm'

new_path = '/home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/'
old_path = '/home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a/'

new = ld.get_multiyr_aflx( 'US-'+site, new_path, gapfilled=True, startyear=start, endyear=end)
new_wg = ld.get_multiyr_aflx( 'US-'+site, new_path, gapfilled=False, startyear=start, endyear=end)
old = ld.get_multiyr_aflx( 'US-'+site, old_path, gapfilled=True, startyear=start, endyear=end)
old_wg = ld.get_multiyr_aflx( 'US-'+site, old_path, gapfilled=False, startyear=start, endyear=end)

Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2007_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2008_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2009_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2010_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2011_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2012_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2013_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2014_gapfilled.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2007_with_gaps.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015_a_rev1/US-Vcm_2008_with_gaps.txt
Parsing /home/greg/sftp/eddyflux/Ameriflux_files/FLUXNET2015

In [109]:
# If desired, plot only particular years
yridx = new.index.year < 2014

## Check flux filtering

Plot the non-gapfilled flux data between for old and new versions of the files, and highlight the differences between them

In [110]:
def iterplot(df1, df2, varname, figref, axnum, idx):
    # Make subplot sharing x axis
    if axnum > 0:
        ax = figref.add_subplot(4,1,axnum+1, sharex=figref.axes[0])
    else:
        ax = figref.add_subplot(4,1,axnum+1)
    # Plot old and new data
    h1 = ax.plot( df1.index[idx], df1.loc[idx, varname], marker='o',
                 mew=0.2, color='blue', ls='none', fillstyle='none')
    h2 = ax.plot( df2.index[idx], df2.loc[idx, varname], '.k')
    # Highlight the changed data points between old and new
    diff = df2[varname]!=df1[varname]
    diff = np.logical_and(idx, diff)
    h3 = ax.plot( df2.index[diff], df2.loc[diff, varname], marker='o',
                 mew=0.2, color='GoldenRod', ms=5, ls='none', fillstyle='none')
    plt.ylabel(varname)

# Now loop through variables and plot
fig0 = plt.figure( figsize=( 16.5, 10.5 ), dpi=100)
varlist = ['FC', 'GPP', 'H', 'LE']
for i, name in enumerate(varlist):
    iterplot(old_wg, new_wg, name, fig0, i, yridx)

fig0.axes[0].legend( ['Old', 'New', 'Changed'], loc='upper right', ncol=3)


## Check gapfilling

Plot the gapfilled flux data between for old and new versions of the files, and highlight the differences between them

In [111]:
def iterplot2(df1, df2, varname, figref, axnum, idx):
    # Make subplot sharing x axis
    if axnum > 0:
        ax = figref.add_subplot(4,1,axnum+1, sharex=figref.axes[0])
    else:
        ax = figref.add_subplot(4,1,axnum+1)
    # Plot old and new data
    h1 = ax.plot( df1.index[idx], df1.loc[idx, varname], marker='o',
                 mew=0.2, color='blue', ls='none', fillstyle='none')
    idx_gf = np.logical_and( idx, df1[varname+'_FLAG'])
    h1gf = ax.plot( df1.index[idx_gf], df1.loc[idx_gf, varname], marker='o',
                 mew=0.2, color='red', ls='none', fillstyle='none')
    h2 = ax.plot( df2.index[idx], df2.loc[idx, varname], '.k')
    idx_gf = np.logical_and( idx, df2[varname+'_FLAG'])
    h2gf = ax.plot( df2.index[idx_gf], df2.loc[idx_gf, varname], '.r')
    # Highlight the changed data points between old and new
    diff = df2[varname]!=df1[varname]
    diff = np.logical_and(idx, diff)
    h3 = ax.plot( df2.index[diff], df2.loc[diff, varname], marker='o',
                 mew=0.2, color='GoldenRod', ms=5, ls='none', fillstyle='none')
    plt.ylabel(varname)

# Now loop through variables and plot
fig1 = plt.figure( figsize=( 16.5, 10.5 ), dpi=100)
varlist = ['FC_F', 'TA_F', 'H_F', 'LE_F']
for i, name in enumerate(varlist):
    iterplot2(old, new, name, fig1, i, yridx)

fig1.axes[0].legend( ['Old', 'Old-GF', 'New', 'New-GF', 'GF-Changed'], loc='upper right', ncol=3)

## Check for USTAR filter

In [120]:
varnm = 'GPP'

figX = plt.figure( figsize=( 16.5, 10.5 ), dpi=100)
ax1 = figX.add_subplot(121)
h1 = ax1.scatter( new_wg.loc[yridx,'USTAR'], new_wg.loc[yridx,varnm], marker='o', color='0.7')
h2 = ax1.scatter( old_wg.loc[yridx,'USTAR'], old_wg.loc[yridx,varnm], marker='.', color='black' )
plt.ylabel('Fc (umol m-2 s-1)')
plt.xlabel('u*')
plt.title('All values')
plt.legend( ['New', 'Old'], loc='lower right' )

# Also plot nighttime values
idx_new = np.logical_or(new_wg.index.hour < 5, new_wg.index.hour > 20)
idx_old = np.logical_or(old_wg.index.hour < 5, old_wg.index.hour > 20)
idx_new = np.logical_and(idx_new, yridx)
idx_old = np.logical_and(idx_old, yridx)

ax2 = figX.add_subplot(122)
h3 = ax2.scatter( new_wg.loc[idx_new,'USTAR'], new_wg.loc[idx_new,varnm], marker='o', color='0.7')
h4 = ax2.scatter( old_wg.loc[idx_old,'USTAR'], old_wg.loc[idx_old,varnm], marker='.', color='black' )
plt.xlabel('u*')
plt.ylabel('Fc (umol m-2 s-1)')
plt.title('Nighttime values')
plt.legend( ['New', 'Old'], loc='lower right' )

## Check CO2 filter

In [113]:
figX = plt.figure( figsize=( 16.5, 10.5 ), dpi=100)
ax1 = figX.add_subplot(121)
h1 = ax1.scatter( new_wg.loc[yridx,'CO2'], new_wg.loc[yridx,'FC'], marker='o', color='0.7')
h2 = ax1.scatter( old_wg.loc[yridx,'CO2'], old_wg.loc[yridx,'FC'], marker='.', color='black' )
plt.ylabel('Fc (umol m-2 s-1)')
plt.xlabel('[CO2]')
plt.title('All values')
plt.legend( ['New', 'Old'], loc='lower right' )

# Also plot nighttime values
idx_new = np.logical_or(new_wg.index.hour < 5, new_wg.index.hour > 20)
idx_old = np.logical_or(old_wg.index.hour < 5, old_wg.index.hour > 20)
idx_new = np.logical_and(idx_new, yridx)
idx_old = np.logical_and(idx_old, yridx)

ax2 = figX.add_subplot(122)
h3 = ax2.scatter( new_wg.loc[idx_new,'CO2'], new_wg.loc[idx_new,'FC'], marker='o', color='0.7')
h4 = ax2.scatter( old_wg.loc[idx_old,'CO2'], old_wg.loc[idx_old,'FC'], marker='.', color='black' )
plt.ylabel('Fc (umol m-2 s-1)')
plt.xlabel('[CO2]')
plt.title('Nighttime values')
plt.legend( ['New', 'Old'], loc='lower right' )

### Plot hourly diagnostics (not gapfilled versions)

In [81]:
# Select flux variable to plot
var = 'FC'

fig1 = plt.figure( figsize=( 11.5, 8.5 ), dpi=100, facecolor='w', edgecolor='k' )
ax1 = fig1.add_subplot(2,3,(1,3))
h1 = ax1.plot(old_wg.index, old_wg[var] - new_wg[var])
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')

ax2 = fig1.add_subplot(2,3,4)
h1 = ax2.scatter( old_wg[var], new_wg[var] )
plt.ylabel('New ' + var + ' (umol m-2 s-1)')
plt.xlabel('Old ' + var + ' (umol m-2 s-1)')

ax3 = fig1.add_subplot(2,3,5)
h1 = ax3.scatter( old_wg.USTAR, old_wg[var] - new_wg[var] )
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')
plt.xlabel('USTAR')

ax4 = fig1.add_subplot(2,3,6)
h1 = ax4.scatter( old_wg.index.time, old_wg[var] - new_wg[var] )
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')
plt.xlabel('time of day')
plt.xlim([0, 24*60*60])
plt.xticks(rotation=70)
#plt.legend( ['New', 'Old'], loc='lower right' )

(array([     0.,  10000.,  20000.,  30000.,  40000.,  50000.,  60000.,
         70000.,  80000.,  90000.]), <a list of 10 Text xticklabel objects>)

### Plot hourly diagnostics - gapfilled versions

In [78]:
# Select flux variable to plot
var = 'FC_F'

fig1 = plt.figure( figsize=( 11.5, 8.5 ), dpi=100, facecolor='w', edgecolor='k' )
ax1 = fig1.add_subplot(2,3,(1,3))
h1 = ax1.plot(old.index, old[var] - new[var])
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')

ax2 = fig1.add_subplot(2,3,4)
h1 = ax2.scatter( old[var], new[var] )
plt.ylabel('New ' + var + ' (umol m-2 s-1)')
plt.xlabel('Old ' + var + ' (umol m-2 s-1)')

ax3 = fig1.add_subplot(2,3,5)
h1 = ax3.scatter( old.USTAR, old[var] - new[var] )
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')
plt.xlabel('USTAR')

ax4 = fig1.add_subplot(2,3,6)
h1 = ax4.scatter( old.index.time, old[var] - new[var] )
plt.ylabel('Residual ' + var + ' (umol m-2 s-1)')
plt.xlabel('time of day')
plt.xlim([0, 24*60*60])
plt.xticks(rotation=70)
#plt.legend( ['New', 'Old'], loc='lower right' )

(array([     0.,  10000.,  20000.,  30000.,  40000.,  50000.,  60000.,
         70000.,  80000.,  90000.]), <a list of 10 Text xticklabel objects>)

## Daily data

In [82]:
new_path = '~/data/current/NMEG_utils/processed_data/daily_aflx/FLUXNET2015_a_rev1'
old_path = '~/data/current/NMEG_utils/processed_data/daily_aflx/FLUXNET2015_litvakustar/'

# Load all sites (its fast anyway)
sites = ['Seg', 'Ses', 'Sen', 'Wjs', 'Mpj', 'Mpg', 'Vcp', 'Vcm']

# New data
daily = { x : 
         ld.load_local_file( new_path + 'US-' + x + '_daily_aflx.csv')
         for x in sites }
d = pd.Panel(daily)
# Old data
daily_old = { x : 
         ld.load_local_file( old_path + 'US-' + x + '_daily_aflx.csv')
         for x in sites }
d_old = pd.Panel(daily_old)

Parsing ~/data/current/NMEG_utils/processed_data/daily_aflx/US-Seg_daily_aflx.csv


OSError: File b'/home/greg/data/current/NMEG_utils/processed_data/daily_aflx/US-Seg_daily_aflx.csv' does not exist

## Plot daily timeseries

In [38]:
# Load for one site
#site = 'Seg'

fig0 = plt.figure( figsize=( 16.5, 10.5 ), dpi=100, facecolor='w', edgecolor='k' )
ax1 = fig0.add_subplot(411)
h1 = ax1.plot( d[site].index, d[site].FC_F_g_int ) 
h2 = ax1.plot( d_old[site].index, d_old[site].FC_F_g_int )
plt.ylabel('Fc (umol m-2)')
plt.legend( ['New', 'Old'], loc='lower right' )

ax2 = fig0.add_subplot(412)
h1 = ax2.plot( d[site].index, d[site].GPP_g_int ) 
h2 = ax2.plot( d_old[site].index, d_old[site].GPP_g_int )
plt.ylabel('GPP (umol m-2)')

ax3 = fig0.add_subplot(413)
h1 = ax3.plot( d[site].index, d[site].RECO_g_int ) 
h2 = ax3.plot( d_old[site].index, d_old[site].RECO_g_int )
plt.ylabel('RECO (umol m-2)')

ax4 = fig0.add_subplot(414)
h1 = ax4.plot( d[site].index, d[site].ET_mm_24hint_0 ) 
h2 = ax4.plot( d_old[site].index, d_old[site].ET_mm_24hint_0 )
plt.ylabel('ET (mm m-2)')

## Annual cumulative fluxes

In [ ]:
# Create dictionaries with modified climatology (see mod_clim)
# for each site
fc_clim = { s : tr.var_climatology( d[s].FC_F_g_int ) for s in sites}
gpp_clim = { s : tr.var_climatology( d[s].GPP_g_int ) for s in sites}
re_clim = { s :tr.var_climatology( d[s].RECO_g_int ) for s in sites}
et_clim = { s : tr.var_climatology( d[s].ET_mm_24hint_0 ) for s in sites}

fc_clim_old = { s : tr.var_climatology( d_old[s].FC_F_g_int ) for s in sites}
gpp_clim_old = { s : tr.var_climatology( d_old[s].GPP_g_int ) for s in sites}
re_clim_old = { s :tr.var_climatology( d_old[s].RECO_g_int ) for s in sites}
et_clim_old = { s : tr.var_climatology( d_old[s].ET_mm_24hint_0 ) for s in sites}

# Function to make an annual table from a climatology
def climtable(clim, sitelist):
    # Make a table
    indices = ['2007', '2008', '2009', '2010', '2011', '2012', '2013',
               '2014', '2015', 'allyr_mean', 'allyr_stdev']
    tbl = pd.DataFrame(columns=sitelist, index = indices)
    # Sum up the climatology columns for the site
    for site in sitelist:
        sums = clim[site].sum()
        # Add each sum to correct site column
        for j in indices:
            tbl.loc[j, site] = sums[j]
    
    return tbl

### NEE, new vs old

In [40]:
nee_t = climtable(fc_clim, sites)
nee_t

,Seg,Ses,Sen,Wjs,Mpj,Mpg,Vcp,Vcm
2007,51.4881,-29.7157,0,-96.5354,0,0,-394.023,-377.925
2008,69.8018,-36.6341,0,-74.8164,-135.507,0,-420.586,-315.194
2009,89.0585,-38.027,0,-121.694,-132.843,-202.578,-381.093,-246.658
2010,9.56343,-50.2597,-49.6294,-147.215,-212.245,-130.636,-365.343,-320.386
2011,108.987,10.2686,139.241,-58.8434,-83.3371,-57.1849,-140.974,-203.534
2012,63.9805,-23.4084,25.2057,-97.2082,-142.823,-95.8503,-172.262,-338.576
2013,4.97954,-40.6457,-78.5021,45.4419,-72.439,36.2029,-305.803,-114.654
2014,41.3959,-12.4544,-48.2824,-80.7155,-27.1398,-47.6662,-266.161,115.599
2015,80.3684,46.093,46.0441,-131.263,-82.194,-90.7524,-177.131,-109.104
allyr_mean,57.8168,-19.4811,5.71083,-89.011,-111.041,-83.7158,-290.809,-211.853


In [41]:
nee_old_t = climtable(fc_clim_old, sites)
nee_old_t

,Seg,Ses,Sen,Wjs,Mpj,Mpg,Vcp,Vcm
2007,53.6465,-39.4861,0,-89.195,0,0,-408.243,-429.677
2008,72.2512,-39.2138,0,-111.502,-143.56,0,-402.926,-338.389
2009,91.2814,-46.6966,0,-114.925,-141.269,-173.007,-383.446,-268.928
2010,6.16041,-56.2854,-61.6386,-164.493,-223.405,-137.871,-396.272,-347.93
2011,110.504,8.82576,111.148,-82.2557,-95.0323,-63.5299,-114.272,-216.773
2012,61.5254,-29.5376,23.066,-112.766,-157.709,-106.398,-149.749,-363.522
2013,6.63731,-44.9694,-84.6501,39.6621,-89.1845,25.5181,-240.795,-112.636
2014,39.7778,-23.2727,-56.6644,-83.1093,-45.149,-55.021,-273.625,94.9486
2015,77.2772,38.0971,36.3786,-138.491,-87.2163,-107.094,-135.83,-144.665
allyr_mean,57.7426,-25.902,-5.36149,-99.5682,-122.769,-87.8415,-277.541,-235.965


### ET, new vs old

In [42]:
et_t = climtable(et_clim, sites)
et_t

,Seg,Ses,Sen,Wjs,Mpj,Mpg,Vcp,Vcm
2007,321.083,302.711,0,338.431,0,0,724.701,633.933
2008,264.924,276.279,0,368.589,334.834,0,600.376,583.692
2009,229.137,217.606,0,277.432,313.95,319.247,580.259,542.248
2010,255.582,230.588,234.237,376.593,401.358,389.743,609.637,519.473
2011,154.073,152.799,124.693,236.215,274.506,252.362,510.384,455.999
2012,205.048,180.731,195.085,342.593,297.507,262.063,541.085,479.817
2013,250.466,223.53,234.543,334.231,328.484,345.667,567.516,276.941
2014,285.654,256.741,240.45,468.755,480.435,446.499,594.565,420.537
2015,268.956,305.449,266.012,592.847,540.286,512.146,737.738,547.771
allyr_mean,248.438,238.597,215.939,379.027,371.716,361.781,607.52,495.94


In [43]:
et_old_t = climtable(et_clim_old, sites)
et_old_t

,Seg,Ses,Sen,Wjs,Mpj,Mpg,Vcp,Vcm
2007,321.926,304.84,0,342.414,0,0,780.188,648.867
2008,266.02,280.335,0,374.534,340.657,0,643.63,592.183
2009,230.569,221.451,0,280.718,318.546,321.457,609.766,549.235
2010,256.654,233.981,235.167,379.911,408.226,392.691,650.056,524.386
2011,154.967,156.371,125.607,239.764,279.036,256.489,542.341,457.447
2012,205.693,183.291,196.195,347.571,301.489,264.297,576.069,481.135
2013,251.333,226.454,235.509,341.148,334.791,349.513,607.654,277.895
2014,287.061,259.922,241.674,474.387,489.027,450.374,640.135,429.478
2015,270.956,312.135,267.265,601.431,551.62,518.865,797.159,558.261
allyr_mean,249.577,242.192,217.008,384.297,378.231,365.49,649.894,502.475
